#### Load project into backend

In [1]:
from pathlib import Path

from mynd.utils.log import logger
from mynd.utils.result import Result

import mynd.backend.metashape as backend

PROJECT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_dev/r23685bc_lite_version.psz"
)
load_result: Result[str, str] = backend.load_project(PROJECT_PATH)

if load_result.is_err():
    logger.error(load_result.err())

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


LoadProject: path = /data/kingston_snv_01/acfr_revisits_metashape_dev/r23685bc_lite_version.psz
loaded project in 21.9635 sec


#### Request camera data from backend

In [4]:
from typing import NamedTuple, Optional

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as ps
import polars as pl

from mynd.api import GroupID, CameraIndexGroup, CameraReferenceGroup
from mynd.utils.result import Ok, Err, Result


def tabulate_camera_references(references: CameraReferenceGroup) -> pl.DataFrame:
    """Creates a data frame / table for aligned cameras."""
    # key, label, x, y, z, roll, pitch, heading

    rows: list[dict] = list()
    for key in bundle.aligned_locations:

        aligned_location: np.ndarray = bundle.aligned_locations.get(key)
        aligned_rotation: np.ndarray = bundle.aligned_rotations.get(key)

        prior_location: np.ndarray = bundle.prior_locations.get(key)
        prior_rotation: np.ndarray = bundle.prior_rotations.get(key)

        row: dict = {
            "key": key,
            "label": label,
            "sensor": sensor,
            "image": image,
            "aligned_locx": aligned_location[0],
            "aligned_locy": aligned_location[1],
            "aligned_locz": aligned_location[2],
            "aligned_rotx": aligned_rotation[0],
            "aligned_roty": aligned_rotation[1],
            "aligned_rotz": aligned_rotation[2],
        }

        if not prior_location is None and not prior_rotation is None:
            row.update(
                {
                    "prior_locx": prior_location[0],
                    "prior_locy": prior_location[1],
                    "prior_locz": prior_location[2],
                    "prior_rotx": prior_rotation[0],
                    "prior_roty": prior_rotation[1],
                    "prior_rotz": prior_rotation[2],
                }
            )

        rows.append(row)

    return pl.DataFrame(rows)


def plot_global_references(
    data: pl.DataFrame,
    lat: str,
    lon: str,
    color: str,
    hover_data: Optional[list] = None,
) -> go.Figure:
    """Plots references in a global coordinate system with map background."""
    fig = px.scatter_mapbox(
        data,
        lat=lat,
        lon=lon,
        color=color,
        size_max=15,
        zoom=16,
        height=600,
        hover_data=hover_data,
    )
    fig.update_layout(height=500, width=600, mapbox_style="open-street-map")
    return fig


def plot_local_references(
    data: pl.DataFrame, x: str, y: str, color: str, hover_data: Optional[list] = None
) -> go.Figure:
    """Plots references in a local coordinate system."""
    fig = px.scatter(data, x=x, y=y, color=color, hover_data=hover_data)
    fig.update_layout(height=800, width=1000)
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    return fig


def on_camera_indices(index_groups: dict[GroupID, CameraIndexGroup]) -> None:
    """TODO"""
    raise NotImplementedError("on_cameras is not implemented")


def on_camera_references(reference_groups: dict[GroupID, CameraReferenceGroup]) -> None:
    """TODO"""
    logger.info("References:")
    for identifier, references in reference_groups.items():
        logger.info(f" - Identifier: {identifier.key}, {identifier.label}")
    raise NotImplementedError("on_camera_references is not implemented")


class CameraData(NamedTuple):
    """Class representing a collection of camera data."""

    indices: CameraIndexGroup
    references: CameraReferenceGroup


def main() -> None:
    """Entrypoint."""

    # Request data from the backend
    index_result: Result = backend.get_camera_indices()
    reference_result: Result = backend.get_camera_references()

    index_groups: dict[GroupID, CameraIndexGroup] = index_result.ok()
    reference_groups: dict[GroupID, CameraReferenceGroup] = index_result.ok()

    logger.info("Indices:")
    for identifier, indices in index_groups.items():
        logger.info(f" - Identifier: {identifier.key}, {identifier.label}")

    logger.info("References:")
    for identifier, references in reference_groups.items():
        logger.info(f" - Identifier: {identifier.key}, {identifier.label}")

    """
    if camera_result.is_err():
        logger.error(camera_request_result.err())

    camera_bundles: dict[Identifier, CameraBundle] = camera_request_result.ok()

    for identifier, bundle in list(camera_bundles.items()):
        camera_data: pl.DataFrame = tabulate_aligned_cameras(bundle)

        figures: dict[str, go.Figure] = {
            "aligned": plot_global_references(
                camera_data,
                lat="aligned_locy",
                lon="aligned_locx",
                color="aligned_locz",
                hover_data=[
                    "key",
                    "label",
                    "sensor",
                    "image",
                    "aligned_rotx",
                    "aligned_roty",
                    "aligned_rotz",
                ],
            ),
            "prior": plot_global_references(
                camera_data,
                lat="prior_locy",
                lon="prior_locx",
                color="prior_locz",
                hover_data=[
                    "key",
                    "label",
                    "sensor",
                    "image",
                    "prior_rotx",
                    "prior_roty",
                    "prior_rotz",
                ],
            ),
        }

        figures.get("aligned").show()
        figures.get("prior").show()

        break
        """


# INVOKE MAIN
main()

2024-09-18 16:38:28.200 | INFO     | __main__:main:116 - Indices:
2024-09-18 16:38:28.201 | INFO     | __main__:main:118 -  - Identifier: 0, r23685bc_20100605_021022
2024-09-18 16:38:28.201 | INFO     | __main__:main:118 -  - Identifier: 1, r23685bc_20120530_233021
2024-09-18 16:38:28.202 | INFO     | __main__:main:118 -  - Identifier: 2, r23685bc_20140616_225022
2024-09-18 16:38:28.202 | INFO     | __main__:main:120 - References:
2024-09-18 16:38:28.202 | INFO     | __main__:main:122 -  - Identifier: 0, r23685bc_20100605_021022
2024-09-18 16:38:28.202 | INFO     | __main__:main:122 -  - Identifier: 1, r23685bc_20120530_233021
2024-09-18 16:38:28.202 | INFO     | __main__:main:122 -  - Identifier: 2, r23685bc_20140616_225022
